In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
%run 'func_prev_comex_ncm.ipynb'

prev_comex_ncm(comex_ncm,cat_cols,target_cols,md_c,md_r)
RandomForestClassifier as RFC_
RandomForestRegressor as RFR_


In [2]:
comex_ncm=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])

# Filtra top
top_nick=comex_ncm.groupby('nick').KG_LIQUIDO.sum().sort_values().index[-2:]
top_uf=comex_ncm.groupby('SG_UF_NCM').KG_LIQUIDO.sum().sort_values().index[-2:]
top_pais=comex_ncm.groupby('CO_PAIS').KG_LIQUIDO.sum().sort_values().index[-2:]
top_urf=comex_ncm.groupby('CO_URF').KG_LIQUIDO.sum().sort_values().index[-2:]
comex_ncm=comex_ncm[comex_ncm.SG_UF_NCM.isin(top_uf)]
comex_ncm=comex_ncm[(comex_ncm.nick.isin(top_nick))]
comex_ncm=comex_ncm[comex_ncm.CO_URF.isin(top_urf)]
comex_ncm=comex_ncm[comex_ncm.CO_PAIS.isin(top_pais)]

In [3]:
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')

In [4]:
#sns.relplot(data=comex_ncm,x='date',y='KG_LIQUIDO',col='nick').fig.suptitle('Importacoes Selecionadas')

# ATENCAO

In [5]:
comex_ncm=comex_ncm[comex_ncm.CO_ANO>2008]

In [6]:
# Define parametros
cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']
l=15
target_cols=['oc','KG_LIQUIDO']
md_c=15
md_r=1

In [7]:
comex_ncm.date=comex_ncm.date.dt.floor('D')

# Confere comprimento em anos
comex_ncm=comex_ncm[comex_ncm.date>comex_ncm.date.max() - pd.DateOffset(years=l)]
(comex_ncm.date.max()-comex_ncm.date.min()).days/365 +1>l

True

In [8]:
repetitions=pd.DataFrame()
for rept in range(5):
    # Preve
    test_pred=prev_comex_ncm(comex_ncm,cat_cols=cat_cols,target_cols=target_cols,md_c=md_c,md_r=md_r)
    
    test_pred=test_pred.assign(cat_cols=str(cat_cols)).assign(target_cols=str(target_cols)).assign(md_c=md_c)\
    .assign(md_r=md_r).assign(rept=rept)

    repetitions=pd.concat([repetitions,test_pred])
    


In [13]:
scores=pd.DataFrame()
for ano in repetitions.CO_ANO.unique():
    for mes in repetitions.CO_MES.unique():
        for nick in repetitions.nick.unique():
            for uf in repetitions.SG_UF_NCM.unique():
                for pais in repetitions.CO_PAIS.unique():
                    for urf in repetitions.CO_URF.unique():
                        repetitions_t=repetitions[
                        (repetitions.CO_ANO==ano)&
                        (repetitions.CO_MES==mes)&
                        (repetitions.nick==nick)&
                        (repetitions.SG_UF_NCM==uf)&
                        (repetitions.CO_PAIS==pais)&
                        (repetitions.CO_URF==urf)
                        ]
                        
                        q=pd.DataFrame({
                            'accuracy_score':[metrics.accuracy_score(repetitions_t.oc,repetitions_t.oc_pred)],
                            'precision_score':[metrics.precision_score(repetitions_t.oc,repetitions_t.oc_pred)],
                            'recall_score':[metrics.recall_score(repetitions_t.oc,repetitions_t.oc_pred)],
                            
                        })
                        
                        q=q.assign(CO_ANO=ano).assign(CO_MES=mes).assign(nick=nick).assign(SG_UF_NCM=uf).assign(CO_PAIS=pais)\
                        .assign(CO_URF=urf)
                        
                        scores=pd.concat([scores,q])
                        
scores.dropna()

,accuracy_score,precision_score,recall_score,CO_ANO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF
0,1.0,1.0,1.0,2024,4,MOP_k,PR,676,917800
0,1.0,1.0,1.0,2024,4,MOP_k,PR,999,917800
0,1.0,1.0,1.0,2024,4,UREIA_k,PR,676,917800
0,0.6,0.0,0.0,2024,4,UREIA_k,PR,676,817800
0,1.0,1.0,1.0,2024,4,UREIA_k,PR,999,917800
0,0.0,0.0,0.0,2024,4,UREIA_k,RS,676,917800
0,1.0,1.0,1.0,2024,4,UREIA_k,RS,999,917800
0,1.0,1.0,1.0,2009,5,UREIA_k,PR,676,917800
